In [ ]:
!pip install pandas sqlalchemy pymysql psycopg2-binary


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 34.1 MB/s eta 0:00:00


In [5]:
from google.colab import files
uploaded = files.upload()   # choose Luxury_Housing_Bangalore.csv


Saving Luxury_Housing_Bangalore.csv to Luxury_Housing_Bangalore.csv


In [6]:
import pandas as pd
df = pd.read_csv("Luxury_Housing_Bangalore.csv")
df.head()


,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4bhk,12.750846039118798,Primary,NRI,2025-03-31,7.990091,5.462863,Launch,Broker,yes,9.212491,18,Loved the amenities!
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3Bhk,16.292151871065954,Primary,Other,2024-06-30,4.839024,NaN,Under construction,NRI Desk,no,7.723898,106,NaN
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4bhk,10.517724412961911,Primary,HNI,2023-12-31,8.131315,8.669227,Ready to move,Direct,yes,6.985493,113,Agent was not responsive.
3,PROP000004,bellary road,Project_3,Embassy,6192.0,3BHK,9.396367494232896,Primary,HNI,2024-03-31,7.501657,5.720246,Ready to move,Online,yes,6.100929,106,Excellent location!
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4Bhk,15.345392444511946,Secondary,HNI,2024-12-31,4.525216,8.609649,Under construction,Broker,no,5.312510,18,Too far from my office.


In [7]:
#  ETL FUNCTION for Luxury Housing Project (Colab)
# -------------------------------------------------------
# Cleans raw CSV, performs feature engineering, and optionally loads to SQL DB.

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime
import re, os

def clean_and_load(csv_path, db_url=None, table_name="realestate_clean"):
    """Clean housing dataset and optionally load to SQL DB."""

    print("📂 Loading CSV:", csv_path)
    df = pd.read_csv(csv_path, low_memory=False)
    print("✅ Raw shape:", df.shape)

    # --------------------- Cleaning ---------------------
    df.columns = [c.strip() for c in df.columns]

    # Normalize text fields
    def normalize_text(s):
        if pd.isna(s):
            return np.nan
        return " ".join(str(s).strip().title().split())

    for col in ["Builder", "Micro_Market", "Configuration", "Sales_Channel",
                "Buyer_Type", "Possession_Status", "Booking_Status"]:
        if col in df.columns:
            df[col] = df[col].astype(str).replace("nan", np.nan)
            df[col] = df[col].apply(normalize_text)

    # Convert Ticket_Price_Cr to numeric INR
    def safe_float(x):
        if pd.isna(x):
            return np.nan
        s = str(x).strip().lower().replace("₹", "").replace(",", "")
        if "cr" in s:
            try:
                return float(s.replace("cr", "").strip()) * 1e7
            except:
                return np.nan
        if "lakh" in s or "lac" in s:
            try:
                s2 = re.sub(r"[^\d\.]", "", s)
                return float(s2) * 1e5
            except:
                return np.nan
        try:
            return float(re.sub(r"[^\d\.]", "", s))
        except:
            return np.nan

    if "Ticket_Price_Cr" in df.columns:
        df["Ticket_Price_Rs"] = df["Ticket_Price_Cr"].apply(safe_float)
    else:
        df["Ticket_Price_Rs"] = np.nan

    # Identify area column and compute price per sqft
    area_col = None
    for ac in ["Carpet_Area_Sqft", "Area_Sqft", "Super_Builtup_Sqft", "Area_Sq.Ft"]:
        if ac in df.columns:
            area_col = ac
            break

    if area_col:
        df[area_col] = pd.to_numeric(df[area_col], errors="coerce")
        df["Price_per_Sqft"] = df["Ticket_Price_Rs"] / df[area_col]
    else:
        df["Price_per_Sqft"] = np.nan

    # Amenity_Score cleaning
    if "Amenity_Score" in df.columns:
        df["Amenity_Score"] = pd.to_numeric(df["Amenity_Score"], errors="coerce")
        if "Micro_Market" in df.columns:
            df["Amenity_Score"] = df.groupby("Micro_Market")["Amenity_Score"].transform(
                lambda x: x.fillna(x.median())
            )
        df["Amenity_Score"] = df["Amenity_Score"].fillna(df["Amenity_Score"].median())

    # Booking_Flag
    if "Booking_Status" in df.columns:
        df["Booking_Flag"] = df["Booking_Status"].apply(
            lambda x: 1 if str(x).lower() in ["booked", "confirmed", "yes", "true", "1"] else 0
        )
    else:
        df["Booking_Flag"] = np.nan

    # Purchase_Quarter → Year and Quarter_Number
    def month_to_quarter(month_int):
        return (int(month_int) - 1) // 3 + 1

    def parse_purchase_quarter(val):
        if pd.isna(val):
            return (np.nan, np.nan)
        s = str(val).strip()
        m = re.search(r"q([1-4])[^\d]*(\d{4})", s, flags=re.I)
        if m:
            return (int(m.group(2)), int(m.group(1)))
        dt = pd.to_datetime(s, errors="coerce")
        if not pd.isna(dt):
            return (dt.year, month_to_quarter(dt.month))
        return (np.nan, np.nan)

    if "Purchase_Quarter" in df.columns:
        parsed = df["Purchase_Quarter"].apply(parse_purchase_quarter)
        df["Purchase_Year"] = parsed.apply(lambda x: x[0] if isinstance(x, tuple) else np.nan)
        df["Quarter_Number"] = parsed.apply(lambda x: x[1] if isinstance(x, tuple) else np.nan)

    # Sentiment from Buyer_Comments
    def simple_sentiment(text):
        if pd.isna(text) or str(text).strip() == "":
            return 0.0
        t = str(text).lower()
        pos = sum(
            1
            for w in [
                "good",
                "great",
                "love",
                "excellent",
                "friendly",
                "yes",
                "interested",
                "likely",
                "positive",
                "affordable",
            ]
            if w in t
        )
        neg = sum(
            1
            for w in [
                "no",
                "not",
                "bad",
                "poor",
                "delay",
                "expensive",
                "complain",
                "negative",
                "dislike",
                "cancel",
            ]
            if w in t
        )
        return float(pos - neg)

    if "Buyer_Comments" in df.columns:
        df["Buyer_Comment_Sentiment"] = df["Buyer_Comments"].apply(simple_sentiment)
    else:
        df["Buyer_Comment_Sentiment"] = 0.0

    # Timestamp
    df["Data_Cleaned_Timestamp"] = datetime.utcnow()

    print("✅ Cleaned shape:", df.shape)

    # --------------------- Save Cleaned Data ---------------------
    clean_path = os.path.join("/content", "realestate_cleaned.csv")
    df.to_csv(clean_path, index=False)
    print(f"💾 Saved cleaned file → {clean_path}")

    # --------------------- Load to SQL (optional) ---------------------
    if db_url:
        print("🗄️ Connecting to DB:", db_url)
        engine = create_engine(db_url)
        df.to_sql(table_name, con=engine, if_exists="replace", index=False, chunksize=5000, method="multi")
        print(f"✅ Loaded into table: {table_name}")
    else:
        print("ℹ️ Skipping SQL load (no db_url provided).")

    print("✨ ETL process completed successfully!")
    return df


In [8]:
df_clean = clean_and_load("Luxury_Housing_Bangalore.csv")


📂 Loading CSV: Luxury_Housing_Bangalore.csv
✅ Raw shape: (101000, 18)


/tmp/ipython-input-1731420230.py:155: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df["Data_Cleaned_Timestamp"] = datetime.utcnow()


✅ Cleaned shape: (101000, 25)
💾 Saved cleaned file → /content/realestate_cleaned.csv
ℹ️ Skipping SQL load (no db_url provided).
✨ ETL process completed successfully!


In [9]:
clean_and_load(
    csv_path="Luxury_Housing_Bangalore.csv",
    db_url=None  # keep None for now
)


📂 Loading CSV: Luxury_Housing_Bangalore.csv
✅ Raw shape: (101000, 18)


/tmp/ipython-input-1731420230.py:155: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  df["Data_Cleaned_Timestamp"] = datetime.utcnow()


✅ Cleaned shape: (101000, 25)
💾 Saved cleaned file → /content/realestate_cleaned.csv
ℹ️ Skipping SQL load (no db_url provided).
✨ ETL process completed successfully!


,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,...,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments,Ticket_Price_Rs,Price_per_Sqft,Booking_Flag,Purchase_Year,Quarter_Number,Buyer_Comment_Sentiment,Data_Cleaned_Timestamp
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4Bhk,12.750846039118798,Primary,Nri,2025-03-31,...,9.212491,18,Loved the amenities!,1.275085e+01,NaN,NaN,2025,1,1.0,2025-10-26 16:57:28.475169
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3Bhk,16.292151871065954,Primary,Other,2024-06-30,...,7.723898,106,NaN,1.629215e+01,NaN,NaN,2024,2,0.0,2025-10-26 16:57:28.475169
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4Bhk,10.517724412961911,Primary,Hni,2023-12-31,...,6.985493,113,Agent was not responsive.,1.051772e+01,NaN,NaN,2023,4,-2.0,2025-10-26 16:57:28.475169
3,PROP000004,Bellary Road,Project_3,Embassy,6192.0,3Bhk,9.396367494232896,Primary,Hni,2024-03-31,...,6.100929,106,Excellent location!,9.396367e+00,NaN,NaN,2024,1,1.0,2025-10-26 16:57:28.475169
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4Bhk,15.345392444511946,Secondary,Hni,2024-12-31,...,5.312510,18,Too far from my office.,1.534539e+01,NaN,NaN,2024,4,0.0,2025-10-26 16:57:28.475169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100995,PROP004730,Bellary Road,Project_229,Embassy,8546.0,5Bhk+,11.33081004147843,Secondary,Cxo,2024-12-31,...,7.511827,22,Will buy after possession.,1.133081e+01,NaN,NaN,2024,4,0.0,2025-10-26 16:57:28.475169
100996,PROP059810,Bellary Road,Project_309,Brigade,3408.0,3Bhk,10.829373158307602,Primary,Cxo,2024-09-30,...,9.851849,26,Agent was not responsive.,1.082937e+01,NaN,NaN,2024,3,-2.0,2025-10-26 16:57:28.475169
100997,PROP065099,Hennur Road,Project_98,RMZ,4691.0,4Bhk,11.183303152058548,Primary,Nri,2023-12-31,...,9.101604,44,Loved the amenities!,1.118330e+01,NaN,NaN,2023,4,1.0,2025-10-26 16:57:28.475169
100998,PROP093022,Rajajinagar,Project_21,Embassy,7435.0,3Bhk,10.914156376035923,Secondary,Other,2024-06-30,...,8.588551,66,Excellent location!,1.091416e+01,NaN,NaN,2024,2,1.0,2025-10-26 16:57:28.475169


In [10]:
import pandas as pd
df_clean = pd.read_csv("realestate_cleaned.csv")
df_clean.head()

,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,...,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments,Ticket_Price_Rs,Price_per_Sqft,Booking_Flag,Purchase_Year,Quarter_Number,Buyer_Comment_Sentiment,Data_Cleaned_Timestamp
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4Bhk,12.750846039118798,Primary,Nri,2025-03-31,...,9.212491,18,Loved the amenities!,12.750846,NaN,NaN,2025,1,1.0,2025-10-26 16:57:28.475169
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3Bhk,16.292151871065954,Primary,Other,2024-06-30,...,7.723898,106,NaN,16.292152,NaN,NaN,2024,2,0.0,2025-10-26 16:57:28.475169
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4Bhk,10.517724412961911,Primary,Hni,2023-12-31,...,6.985493,113,Agent was not responsive.,10.517724,NaN,NaN,2023,4,-2.0,2025-10-26 16:57:28.475169
3,PROP000004,Bellary Road,Project_3,Embassy,6192.0,3Bhk,9.396367494232896,Primary,Hni,2024-03-31,...,6.100929,106,Excellent location!,9.396367,NaN,NaN,2024,1,1.0,2025-10-26 16:57:28.475169
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4Bhk,15.345392444511946,Secondary,Hni,2024-12-31,...,5.312510,18,Too far from my office.,15.345392,NaN,NaN,2024,4,0.0,2025-10-26 16:57:28.475169


In [11]:
import pandas as pd
from textblob import TextBlob
import numpy as np

# 1️⃣ Load the cleaned CSV file
df = pd.read_csv("/content/realestate_cleaned.csv")

print("✅ File loaded successfully!")
print("Before cleaning:", df.shape)
print("Columns before:", list(df.columns))

# 2️⃣ Clean the Ticket_Price_Cr column (remove ₹, commas, 'Cr', spaces)
df['Ticket_Price_Cr'] = (
    df['Ticket_Price_Cr']
    .astype(str)
    .str.replace('₹', '', regex=False)
    .str.replace('Cr', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.strip()
)
df['Ticket_Price_Cr'] = pd.to_numeric(df['Ticket_Price_Cr'], errors='coerce')

# 3️⃣ Ensure numeric columns are clean
for col in ['Amenity_Score', 'Connectivity_Score', 'Locality_Infra_Score', 'Unit_Size_Sqft']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 4️⃣ Derive Booking_Flag (1 = booked)
df['Booking_Flag'] = (
    (df['Possession_Status'].astype(str).str.lower().isin(['ready to move', 'under construction'])) &
    (df['Amenity_Score'] >= 6)
).astype(int)

# 5️⃣ Derive Price_per_Sqft safely
df['Price_per_Sqft'] = np.where(
    df['Unit_Size_Sqft'] > 0,
    (df['Ticket_Price_Cr'] * 10000000) / df['Unit_Size_Sqft'],
    np.nan
)

# 6️⃣ Derive Quarter_Number and Purchase_Year
df['Purchase_Quarter'] = pd.to_datetime(df['Purchase_Quarter'], errors='coerce')
df['Quarter_Number'] = df['Purchase_Quarter'].dt.quarter
df['Purchase_Year'] = df['Purchase_Quarter'].dt.year

# 7️⃣ Derive Buyer_Comment_Sentiment
df['Buyer_Comment_Sentiment'] = df['Buyer_Comments'].fillna('').apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# 8️⃣ Drop unnecessary columns
drop_cols = ['Ticket_Price_Rs', 'Data_Cleaned_Timestamp']
df.drop(columns=drop_cols, inplace=True, errors='ignore')

# 9️⃣ Reorder columns neatly
ordered_cols = [
    'Property_ID', 'Project_Name', 'Developer_Name', 'Micro_Market',
    'Unit_Size_Sqft', 'Configuration', 'Ticket_Price_Cr', 'Price_per_Sqft',
    'Amenity_Score', 'Connectivity_Score', 'Locality_Infra_Score',
    'Possession_Status', 'Booking_Flag', 'Sales_Channel', 'Transaction_Type',
    'Buyer_Type', 'NRI_Buyer', 'Buyer_Comments', 'Buyer_Comment_Sentiment',
    'Purchase_Quarter', 'Purchase_Year', 'Quarter_Number', 'Avg_Traffic_Time_Min'
]
df = df[ordered_cols]

# 🔟 Save the final cleaned and derived file
df.to_csv("/content/cleaned_realestate_final.csv", index=False)
print("✅ Final cleaned dataset saved as cleaned_realestate_final.csv")
print("Columns now:", df.columns.tolist())


✅ File loaded successfully!
Before cleaning: (101000, 25)
Columns before: ['Property_ID', 'Micro_Market', 'Project_Name', 'Developer_Name', 'Unit_Size_Sqft', 'Configuration', 'Ticket_Price_Cr', 'Transaction_Type', 'Buyer_Type', 'Purchase_Quarter', 'Connectivity_Score', 'Amenity_Score', 'Possession_Status', 'Sales_Channel', 'NRI_Buyer', 'Locality_Infra_Score', 'Avg_Traffic_Time_Min', 'Buyer_Comments', 'Ticket_Price_Rs', 'Price_per_Sqft', 'Booking_Flag', 'Purchase_Year', 'Quarter_Number', 'Buyer_Comment_Sentiment', 'Data_Cleaned_Timestamp']
✅ Final cleaned dataset saved as cleaned_realestate_final.csv
Columns now: ['Property_ID', 'Project_Name', 'Developer_Name', 'Micro_Market', 'Unit_Size_Sqft', 'Configuration', 'Ticket_Price_Cr', 'Price_per_Sqft', 'Amenity_Score', 'Connectivity_Score', 'Locality_Infra_Score', 'Possession_Status', 'Booking_Flag', 'Sales_Channel', 'Transaction_Type', 'Buyer_Type', 'NRI_Buyer', 'Buyer_Comments', 'Buyer_Comment_Sentiment', 'Purchase_Quarter', 'Purchase_Yea

In [12]:
print(df.shape)
print(df['Booking_Flag'].value_counts())
df.head()


(101000, 23)
Booking_Flag
1    55344
0    45656
Name: count, dtype: int64


,Property_ID,Project_Name,Developer_Name,Micro_Market,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Price_per_Sqft,Amenity_Score,Connectivity_Score,...,Sales_Channel,Transaction_Type,Buyer_Type,NRI_Buyer,Buyer_Comments,Buyer_Comment_Sentiment,Purchase_Quarter,Purchase_Year,Quarter_Number,Avg_Traffic_Time_Min
0,PROP000001,Project_0,RMZ,Sarjapur Road,4025.0,4Bhk,12.750846,31679.120594,5.462863,7.990091,...,Broker,Primary,Nri,yes,Loved the amenities!,0.875,2025-03-31,2025,1,18
1,PROP000002,Project_1,Puravankara,Indiranagar,5760.0,3Bhk,16.292152,28284.985887,7.521263,4.839024,...,Nri Desk,Primary,Other,no,NaN,0.000,2024-06-30,2024,2,106
2,PROP000003,Project_2,Tata Housing,Bannerghatta Road,7707.0,4Bhk,10.517724,13646.976013,8.669227,8.131315,...,Direct,Primary,Hni,yes,Agent was not responsive.,0.000,2023-12-31,2023,4,113
3,PROP000004,Project_3,Embassy,Bellary Road,6192.0,3Bhk,9.396367,15175.012103,5.720246,7.501657,...,Online,Primary,Hni,yes,Excellent location!,1.000,2024-03-31,2024,1,106
4,PROP000005,Project_4,SNN Raj,Koramangala,7147.0,4Bhk,15.345392,21471.096187,8.609649,4.525216,...,Broker,Secondary,Hni,no,Too far from my office.,0.100,2024-12-31,2024,4,18


In [13]:
# ✨ Round off for better visualization
df['Price_per_Sqft'] = df['Price_per_Sqft'].round(0)
df['Amenity_Score'] = df['Amenity_Score'].round(1)
df['Connectivity_Score'] = df['Connectivity_Score'].round(1)
df['Locality_Infra_Score'] = df['Locality_Infra_Score'].round(1)
df['Buyer_Comment_Sentiment'] = df['Buyer_Comment_Sentiment'].round(2)

print("✅ All key numeric fields rounded for better visualization!")


✅ All key numeric fields rounded for better visualization!


In [14]:
print(df.shape)
print(df['Booking_Flag'].value_counts())
df.head()


(101000, 23)
Booking_Flag
1    55344
0    45656
Name: count, dtype: int64


,Property_ID,Project_Name,Developer_Name,Micro_Market,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Price_per_Sqft,Amenity_Score,Connectivity_Score,...,Sales_Channel,Transaction_Type,Buyer_Type,NRI_Buyer,Buyer_Comments,Buyer_Comment_Sentiment,Purchase_Quarter,Purchase_Year,Quarter_Number,Avg_Traffic_Time_Min
0,PROP000001,Project_0,RMZ,Sarjapur Road,4025.0,4Bhk,12.750846,31679.0,5.5,8.0,...,Broker,Primary,Nri,yes,Loved the amenities!,0.88,2025-03-31,2025,1,18
1,PROP000002,Project_1,Puravankara,Indiranagar,5760.0,3Bhk,16.292152,28285.0,7.5,4.8,...,Nri Desk,Primary,Other,no,NaN,0.00,2024-06-30,2024,2,106
2,PROP000003,Project_2,Tata Housing,Bannerghatta Road,7707.0,4Bhk,10.517724,13647.0,8.7,8.1,...,Direct,Primary,Hni,yes,Agent was not responsive.,0.00,2023-12-31,2023,4,113
3,PROP000004,Project_3,Embassy,Bellary Road,6192.0,3Bhk,9.396367,15175.0,5.7,7.5,...,Online,Primary,Hni,yes,Excellent location!,1.00,2024-03-31,2024,1,106
4,PROP000005,Project_4,SNN Raj,Koramangala,7147.0,4Bhk,15.345392,21471.0,8.6,4.5,...,Broker,Secondary,Hni,no,Too far from my office.,0.10,2024-12-31,2024,4,18


In [ ]:
df.to_csv("/content/cleaned_realestate_final.csv", index=False)


In [ ]:
!pip install sqlalchemy pymysql


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# ✅ Load your final cleaned dataset
csv_path = "/content/cleaned_realestate_final.csv"
df = pd.read_csv(csv_path)
print("✅ Loaded final dataset:", df.shape)

# ✅ MySQL connection details
DB_USER = "harif"
DB_PASS = "root"
DB_HOST = "127.0.0.1"
DB_PORT = "3306"
DB_NAME = "luxury_sales"
TABLE_NAME = "cleaned_realestate_final"

# ✅ Create SQLAlchemy engine
connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

# ✅ Upload data to MySQL
df.to_sql(TABLE_NAME, con=engine, if_exists="replace", index=False, chunksize=1000, method="multi")

print(f"✅ Data successfully loaded into MySQL table: {TABLE_NAME}")
print(f"Database: {DB_NAME}")


In [ ]:
# ==========================================================
# 🏡 Luxury Housing Analytics – Data Cleaning Script (Colab)
# Cleans null, blank, and negative values and derives Price_per_Sqft
# ==========================================================

import pandas as pd
import numpy as np

# ✅ Step 1: Upload your CSV file
from google.colab import files
uploaded = files.upload()

# ✅ Step 2: Load dataset
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)
print("✅ File loaded successfully:", df.shape)

# ✅ Step 3: Check columns
print("\nAvailable columns:\n", df.columns.tolist())

# ✅ Step 4: Clean Ticket_Price_Cr (main price column)
# Replace blank, null, or negative with NaN
df['Ticket_Price_Cr'] = pd.to_numeric(df['Ticket_Price_Cr'], errors='coerce')
df['Ticket_Price_Cr'] = df['Ticket_Price_Cr'].apply(lambda x: np.nan if x <= 0 else x)

# Fill missing values with median per Micro_Market
if 'Micro_Market' in df.columns:
    df['Ticket_Price_Cr'] = df.groupby('Micro_Market')['Ticket_Price_Cr'].transform(
        lambda x: x.fillna(x.median())
    )

# Fill remaining missing prices with global median
df['Ticket_Price_Cr'] = df['Ticket_Price_Cr'].fillna(df['Ticket_Price_Cr'].median())

print("\n✅ Negative / Null Ticket_Price_Cr values cleaned successfully.")

# ✅ Step 5: Clean and derive Price_per_Sqft
# Convert price (Cr → Rupees) and divide by Unit_Size_Sqft
df['Price_per_Sqft'] = np.where(
    (df['Unit_Size_Sqft'] > 0),
    round((df['Ticket_Price_Cr'] * 10000000) / df['Unit_Size_Sqft'], 2),
    np.nan
)

# Fill missing Price_per_Sqft values using median per Micro_Market
if 'Micro_Market' in df.columns:
    df['Price_per_Sqft'] = df.groupby('Micro_Market')['Price_per_Sqft'].transform(
        lambda x: x.fillna(x.median())
    )

# Fill any remaining missing Price_per_Sqft with global median
df['Price_per_Sqft'] = df['Price_per_Sqft'].fillna(df['Price_per_Sqft'].median())

print("✅ Derived and filled Price_per_Sqft successfully.")

# ✅ Step 6: Clean up other empty text columns (optional)
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.fillna("Not Specified")

# ✅ Step 7: Generate Summary
summary = df[['Ticket_Price_Cr', 'Price_per_Sqft']].describe()
print("\n📊 Summary after cleaning:\n", summary)

# ✅ Step 8: Save cleaned dataset
output_file = "cleaned_realestate_final_filled.csv"
df.to_csv(output_file, index=False)
print(f"\n💾 Cleaned dataset saved as: {output_file}")

# ✅ Step 9: Download cleaned file
from google.colab import files
files.download(output_file)
